In [12]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup
import pandas as pd
import cloudscraper
import re
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from datetime import datetime
from webbrowser import Chrome

In [36]:
path = "chromedriver/chromedriver.exe"

chrome_options = webdriver.ChromeOptions()
service = Service(path)

browser = webdriver.Chrome(options=chrome_options, service=service)
browser2 = webdriver.Chrome(options=chrome_options, service=service)

url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
scraper = cloudscraper.create_scraper(delay=10, browser="chrome")
content = scraper.get(url).content

browser.get(url)
content = browser.page_source
soup = BeautifulSoup(content, 'html.parser')
movies = soup.find_all('div',{"class": 'sc-b189961a-0 hBZnfJ cli-children'})

# Mengecek ketersediaan file dan memastikan isinya tidak kosong
filename = "halaman_movies_250.csv"
filename2 = "box_office_250.csv"
filename3 = "technical_specs_250.csv"

# Menghapus file jika sudah ada dan tidak kosong
if os.path.isfile(filename) and os.path.getsize(filename) > 0:
    os.remove(filename)
    os.remove(filename2)
    os.remove(filename3)

# Membuka file untuk menulis
with open(filename, "w", encoding="utf-8") as f:
    fheaders = "Name,Year,Durasi(Menit),Rating\n"
    f.write(fheaders)

with open(filename2, "w", encoding="utf-8") as f2:
    f2headers = "Name,Budget,Gross_US,Opening_Week,Open_Week_Date,Gross_World\n"
    f2.write(f2headers)

with open(filename3, "w", encoding="utf-8") as f3:
    f3headers = "Name,Color,Sound_Mix,Aspect_Ratio\n"
    f3.write(f3headers)

try:
    # Membuka sesi untuk URL pertama
    browser.get(url)

    # Mendapatkan halaman pertama dan mencari data film
    first_page = BeautifulSoup(browser.page_source, 'html.parser')
    movies = first_page.find_all("div", {"class": "sc-b189961a-0 hBZnfJ cli-children"})

    print("Data Halaman Depan")
    print("-----------------------------------")

    movie_titles = []
    movie_data = []

    for movie in movies:
        menit = 0
        rating = "Not Rated"
        name = movie.find("h3", {"class": "ipc-title__text"}).text
        movie_titles.append(name)
        detil_line = movie.findAll("span", {"class": "sc-b189961a-8 kLaxqf cli-title-metadata-item"})
        if len(detil_line) > 2:
            year_movie = detil_line[0].text
            durasi = detil_line[1].text.split(" ")
            if len(durasi) > 1:
                try:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                except ValueError:
                    menit = 0
            else:
                try:
                    menit = int(durasi[0].replace("h", "")) * 60
                except ValueError:
                    menit = 0
            rating = detil_line[2].text
        elif len(detil_line) > 1:
            year_movie = detil_line[0].text
            durasi = detil_line[1].text.split(" ")
            if len(durasi) > 1:
                try:
                    menit = int(durasi[0].replace("h", "")) * 60 + int(durasi[1].replace("m", ""))
                except ValueError:
                    menit = 0
            else:
                try:
                    menit = int(durasi[0].replace("h", "")) * 60
                except ValueError:
                    menit = 0
        else:
            year_movie = detil_line[0].text

        print(name + "," + year_movie + "," + str(menit) + "," + rating)
        movie_data.append(f"{name},{year_movie},{str(menit)},{rating}\n")

    # Menulis data ke file setelah loop
    with open(filename, "a", encoding="utf-8") as f:
        f.writelines(movie_data)

    # Mendapatkan halaman detail dan mencari data film box office dan technical specs
    links = browser.find_elements(By.XPATH, '//div[@class="sc-b189961a-0 hBZnfJ cli-children"]')
    print("-----------------------------------")
    print("Data Box Office dan Technical Specs")
    print("panjang ", len(links))

    box_office_data = []
    tech_specs_data = []

    for count, link in enumerate(links):
        try:
            title = movie_titles[count]
            print("")
            print("Title {}: {}".format(count + 1, title))
            print("-----------------------------------")
            budget_num = 0
            gross_us_num = 0
            open_week_rev_num = 0
            open_week_date_std = "1900-01-01 00:00:00"
            gross_world_num = 0
            color_str = ""
            sound_mix_str = ""
            aspect_ratio = ""

            browser2.get(link.find_element(By.CSS_SELECTOR, "a").get_attribute('href'))

            # BOX OFFICE DATA ON THE 2ND URL
            det_page = browser2.page_source
            container_rows = BeautifulSoup(det_page, "html.parser")
            box_office_elements = container_rows.find("div", {"data-testid": "title-boxoffice-section"})
            tech_specs_elements = container_rows.find("div", {"data-testid": "title-techspecs-section"})

            if box_office_elements is not None:
                det_movie = box_office_elements.find_all("span", {"class": "ipc-metadata-list-item__list-content-item"})
                if len(det_movie) > 4:
                    budget = det_movie[0].text
                    gross_us = det_movie[1].text
                    open_week_rev = det_movie[2].text
                    open_week_date = det_movie[3].text
                    gross_world = det_movie[4].text

                    try:
                        budget_num = int(re.sub("[^0-9]", "", budget))
                    except ValueError:
                        budget_num = 0

                    try:
                        gross_us_num = int(re.sub("[^0-9]", "", gross_us))
                    except ValueError:
                        gross_us_num = 0

                    try:
                        open_week_rev_num = int(re.sub("[^0-9]", "", open_week_rev))
                    except ValueError:
                        open_week_rev_num = 0

                    try:
                        open_week_date_std = datetime.strptime(open_week_date, "%b %d, %Y")
                    except ValueError:
                        open_week_date_std = "1900-01-01 00:00:00"

                    try:
                        gross_world_num = int(re.sub("[^0-9]", "", gross_world))
                    except ValueError:
                        gross_world_num = 0

                    print("Budget: ", budget_num)
                    print("Gross US: ", gross_us_num)
                    print("Opening Week Revenue: ", open_week_rev_num)
                    print("Opening Week Date: ", open_week_date_std)
                    print("Gross World: ", gross_world_num)
                    print(" ")

            if tech_specs_elements is not None:
                tech_specs = tech_specs_elements.find_all("div", {"class": "ipc-metadata-list-item__content-container"})
                if len(tech_specs) > 3:
                    runtime = tech_specs[0].text
                    color = tech_specs[1].text
                    sound_mix = tech_specs[2].text
                    aspect_ratio = tech_specs[3].text

                    # Menggabungkan dan membersihkan data
                    color_items = [item.text for item in tech_specs[1].find_all("a", {"class": "ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"})]
                    sound_mix_items = [item.text for item in tech_specs[2].find_all("a", {"class": "ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"})]

                    color_str = " ".join(color_items)
                    sound_mix_str = " ".join(sound_mix_items)

                    print("Color: ", color_str)
                    print("Sound Mix: ", sound_mix_str)
                    print("Aspect Ratio: ", aspect_ratio)

            box_office_data.append(f"{title},{budget_num},{gross_us_num},{open_week_rev_num},{open_week_date_std},{gross_world_num}\n")
            tech_specs_data.append(f"{title},{color_str},{sound_mix_str},{aspect_ratio}\n")

        except Exception as e:
            print(f"Error processing link: {e}")

    # Menulis data ke file setelah loop
    with open(filename2, "a", encoding="utf-8") as f2:
        f2.writelines(box_office_data)

    with open(filename3, "a", encoding="utf-8") as f3:
        f3.writelines(tech_specs_data)

except Exception as e:
    print(e)

finally:
    browser.quit()
    browser2.quit()


Data Halaman Depan
-----------------------------------
1. The Shawshank Redemption,1994,142,R
2. The Godfather,1972,175,R
3. The Dark Knight,2008,152,PG-13
4. The Godfather Part II,1974,202,R
5. 12 Angry Men,1957,96,Approved
6. Schindler's List,1993,195,R
7. The Lord of the Rings: The Return of the King,2003,201,PG-13
8. Pulp Fiction,1994,154,R
9. The Lord of the Rings: The Fellowship of the Ring,2001,178,PG-13
10. The Good, the Bad and the Ugly,1966,178,R
11. Forrest Gump,1994,142,PG-13
12. The Lord of the Rings: The Two Towers,2002,179,PG-13
13. Fight Club,1999,139,R
14. Inception,2010,148,PG-13
15. Star Wars: Episode V - The Empire Strikes Back,1980,124,PG
16. The Matrix,1999,136,R
17. Goodfellas,1990,145,R
18. One Flew Over the Cuckoo's Nest,1975,133,R
19. Se7en,1995,127,R
20. Interstellar,2014,169,PG-13
21. It's a Wonderful Life,1946,130,PG
22. Seven Samurai,1954,207,Not Rated
23. The Silence of the Lambs,1991,118,R
24. Saving Private Ryan,1998,169,R
25. City of God,2002,130,R
26.

In [39]:
fn1 = 'halaman_movies_250.csv'
fn2 = 'box_office_250.csv'
fn3 = 'technical_specs_250.csv'
fn4 = 'imdb_combined.csv'

# Menghapus file output jika sudah ada
if os.path.isfile(fn4) and os.path.getsize(fn4) > 0:      
    os.remove(fn4)

# Read CSV files into dataframes
df1 = pd.read_csv(fn1)
df2 = pd.read_csv(fn2)
df3 = pd.read_csv(fn3)

merged_df = df1.merge(df2, on='Name', how='inner')

final_merged_df = merged_df.merge(df3, on='Name', how='inner')

final_merged_df.to_csv(fn4, index=False)

In [41]:
data = pd.read_csv('imdb_combined.csv')

# Fungsi untuk menghilangkan angka pada judul
def remove_numbers(name):
    return re.sub(r'^\d+\.\s*', '', name)

# Terapkan fungsi ke kolom "Name"
data['Name'] = data['Name'].apply(remove_numbers)

# Tampilkan beberapa baris pertama untuk memverifikasi perubahan
data.head()

# Menyimpan data ke file CSV
data.to_csv('data/imdb_combined.csv', index=False)